# Gradient Boosted Trees

In [ ]:
from pi_pact_sort import bin_categorize, categorize

# Config
ADDITIONAL_FEATURES = ['HUMIDITY', 'PRESSURE'] # Contains the strings 'HUMIDITY' and/or 'PRESSURE', or none
CATEGORIZE_FUNC = bin_categorize # One of the two binning functions in pi_pact_sort

# Automatically set objective for XGBClassifier and the file string for pickling
if len(ADDITIONAL_FEATURES) > 0:
    if len(ADDITIONAL_FEATURES) == 1:
        if 'HUMIDITY' in ADDITIONAL_FEATURES:
            feature_str = '3varH'
        else:
            feature_str = '3varP'
    else:
        feature_str = '4var'
else:
    feature_str = '2var'

if CATEGORIZE_FUNC == categorize:
    objective = 'multi:softmax'
    num_classes = 3
    metric='merror'
    scoring='accuracy'
    label_str = '3b'
else:
    objective = 'binary:logistic'
    num_classes = None
    metric='auc'
    scoring='roc_auc'
    label_str = 'binary'
    
file_str = f"xgboost-models/{feature_str}-{label_str}-xgboost-model.pickle"

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

import numpy as np
import pandas as pd
from pathlib import Path
import pickle
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

DROP_COLUMNS = ['ADDRESS', 'TIMESTAMP', 'UUID', 'MAJOR', 'MINOR', 'TX POWER', 'TEMPERATURE',
                'PITCH', 'ROLL', 'YAW', 'SCAN']
for feature in ['HUMIDITY', 'PRESSURE']:
    if feature not in ADDITIONAL_FEATURES:
        DROP_COLUMNS.append(feature)
SAMPLE_SIZE = 30000
np.random.seed(0)


"""Trains a Gradient Boosted Trees classifier to predict a distance range given RSSI values and other variables.
"""

# Initialize DataFrame
data: pd.DataFrame = pd.DataFrame(columns=['RSSI', 'DISTANCE'] + ADDITIONAL_FEATURES)
data_copy: pd.DataFrame = data.copy()
csv_file: Path
for csv_file in Path('.').glob('indoor-noObstruct-SenseHat*/*.csv'):
    datapart: pd.DataFrame = pd.read_csv(csv_file)
    for column in DROP_COLUMNS:
        if column in datapart.columns:
            datapart = datapart.drop([column], 1)
    data_copy = data_copy.append(datapart)

# Categorize distance
data_copy['DISTANCE'] = data_copy['DISTANCE'].map(CATEGORIZE_FUNC)

# Sample data from each distance category
for value in data_copy['DISTANCE'].unique():
    datapart = data_copy[data_copy.DISTANCE == value]
    datapart = datapart.sample(SAMPLE_SIZE, random_state=1)
    data = data.append(datapart)

# Assign features and labels
X: np.array = data.drop(['DISTANCE'], 1).to_numpy()
y: np.array = data['DISTANCE'].to_numpy(dtype=int)

Code below adapted from Aarshay Jain's "Complete Guide to Parameter Tuning in XGBoost with codes in Python", the article can be found at:`https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/` and the full code from the article can be found at: `https://github.com/aarshayj/analytics_vidhya/blob/master/Articles/Parameter_Tuning_XGBoost_with_Example/XGBoost%20models.ipynb`.

In [ ]:
def modelfit(alg, X, y, useTrainCV=True, cv_folds=5, early_stopping_rounds=50) -> int:
    """
    Fits an 'XGBClassifier' instance and provides metrics for hyperparameter tuning.
    
    Args:
        alg (XGBClassifier): the 'XGBClassifier' instance to be fitted.
        X (np.array, shape=(n_samples, n_features)): the feature data.
        y (np.array, shape=(n_samples, )): the corresponding labels.
        useTrainCV (bool): controls whether cross-validation should be used.
        cv_folds (int): the number of folds to use for cross-validation.
        early_stopping_rounds (int): the maximum number of iterations that may occur without an increase in
            cross-validation score before training terminates. This value is passed to the native 'xgb.cv'.
            
    Returns:
        The optimal number of trees given `alg`'s parameters.
    """
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X, label=y, feature_names=['RSSI'] + ADDITIONAL_FEATURES)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'],
                          nfold=cv_folds, metrics=metric, early_stopping_rounds=early_stopping_rounds,
                          verbose_eval=True)
        best_n_estimators = cvresult.shape[0]
        alg.set_params(n_estimators=best_n_estimators)
    
    #Fit the algorithm on the data
    alg.fit(X, y, eval_metric=metric)
        
    #Predict training set:
    dtrain_predictions = alg.predict(X)
       
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % accuracy_score(y, dtrain_predictions))
    if CATEGORIZE_FUNC == bin_categorize:
        dtrain_predprob = alg.predict_proba(X)[:,1]
        print("AUC Score (Train): %f" % roc_auc_score(y, dtrain_predprob))
                    
    feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')
    
    if useTrainCV:
        return best_n_estimators

The first step of Jain's hyperparameter tuning process is to find the optimal number of trees with a fixed learning rate ($\eta=0.1$)

In [ ]:
xgb1 = XGBClassifier(learning_rate =0.1,
                     n_estimators=1000,
                     max_depth=5,
                     min_child_weight=1,
                     gamma=0,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     objective=objective,
                     num_class=num_classes,
                     nthread=4,
                     scale_pos_weight=1,
                     seed=0)
best_n_estimators_1 = modelfit(xgb1, X, y)

Then, we tune the integer parameters `max_depth` and `min_child_weight`.

In [ ]:
param_test1 = {
     'max_depth': np.arange(1, 8, 1),
     'min_child_weight': np.arange(1, 8, 1)
}

gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=best_n_estimators_1,
                                                  max_depth=5, min_child_weight=1, gamma=0, subsample=0.8,
                                                  colsample_bytree=0.8, objective=objective,
                                                  num_class=num_classes, nthread=4, scale_pos_weight=1,
                                                  seed=0), 
                        param_grid = param_test1, scoring=scoring, n_jobs=4, cv=5)
gsearch1.fit(X, y)
best_max_depth = gsearch1.best_params_['max_depth']
best_min_child_weight = gsearch1.best_params_['min_child_weight']
print(gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_)

Next, we tune the `gamma` parameter.

In [ ]:
param_test2 = {
     'gamma':[i/10.0 for i in range(0,5)]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=best_n_estimators_1,
                                                  max_depth=best_max_depth,
                                                  min_child_weight=best_min_child_weight,
                                                  gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective=objective, num_class=num_classes,
                                                  nthread=4, scale_pos_weight=1, seed=0), 
                        param_grid = param_test2, scoring=scoring, n_jobs=4, cv=5)
gsearch2.fit(X, y)
best_gamma = gsearch2.best_params_['gamma']
print(gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_)

Before we proceed, Jain recommends recalibrating the value of `n_estimators` with our new parameters.

In [ ]:
xgb2 = XGBClassifier(learning_rate=0.1,
                     n_estimators=1000,
                     max_depth=best_max_depth,
                     min_child_weight=best_min_child_weight,
                     gamma=best_gamma,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     objective=objective,
                     num_class=num_classes,
                     nthread=4,
                     scale_pos_weight=1,
                     seed=0)
best_n_estimators_2 = modelfit(xgb2, X, y)

Jain's next step is to tune `subsample` and `colsample_bytree`.

In [ ]:
param_test3 = {
     'subsample':[i/100.0 for i in range (75, 85, 1)],
     'colsample_bytree':[i/100.0 for i in range(65, 75, 1)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=best_n_estimators_2,
                                                  max_depth=best_max_depth,
                                                  min_child_weight=best_min_child_weight,
                                                  gamma=best_gamma, subsample=0.8,
                                                  colsample_bytree=0.8,
                                                  objective=objective, num_class=num_classes, 
                                                  nthread=4, scale_pos_weight=1, seed=0), 
                        param_grid = param_test3, scoring=scoring, n_jobs=4, cv=5)
gsearch3.fit(X, y)
best_subsample = gsearch3.best_params_['subsample']
best_colsample_bytree = gsearch3.best_params_['colsample_bytree']
print(gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_)

Then we tune `reg_alpha` and `reg_lambda`.

In [ ]:
param_test4 = {
     'reg_alpha': [0, 1e-8, 1e-6],
     'reg_lamba': [0, ]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=best_n_estimators_2,
                                                  max_depth=best_max_depth,
                                                  min_child_weight=best_min_child_weight,
                                                  gamma=best_gamma, subsample=best_subsample,
                                                  colsample_bytree=best_colsample_bytree,
                                                  objective=objective, num_class=num_classes,
                                                  nthread=4, scale_pos_weight=1, seed=0), 
                        param_grid=param_test4, scoring=scoring, n_jobs=4, cv=5)
gsearch4.fit(X, y)
best_alpha = gsearch4.best_params_['reg_alpha']
best_lambda = gsearch4.best_params_['reg_lamba']
print(gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_)

We finish by reducing $\eta$ and increasing `n_estimators`.

In [ ]:
xgb4 = XGBClassifier(learning_rate=0.01,
                     n_estimators=5000,
                     max_depth=best_max_depth,
                     min_child_weight=best_min_child_weight,
                     gamma=best_gamma,
                     subsample=best_subsample,
                     colsample_bytree=best_colsample_bytree,
                     reg_alpha=best_alpha,
                     reg_lamba=best_lambda,
                     objective=objective,
                     num_class=num_classes,
                     nthread=4,
                     scale_pos_weight=1,
                     seed=0)
modelfit(xgb4, X, y)

In [ ]:
# Print tuned parameters
print(f"Max depth: {best_max_depth}\n"
      f"Min child weight: {best_min_child_weight}\n"
      f"Gamma: {best_gamma}\n"
      f"Colsample by tree: {best_colsample_bytree}\n"
      f"Subsample: {best_subsample}\n"
      f"Reg alpha: {best_alpha}\n"
      f"Reg lamba: {best_lambda}")

In [ ]:
# Pickle model for future use
with open(file_str, "wb") as f:
    pickle.dump(xgb4, f)

In [ ]:
from sklearn.metrics import roc_curve

# Save an roc curve if using binary classification
if CATEGORIZE_FUNC == bin_categorize:
    probs = xgb4.predict_proba(X)
    fpr, tpr, _ = roc_curve(y, probs[:, 1])
    plt.plot(fpr, tpr)
    plt.title('ROC curve')
    plt.xlabel('false positive rate')
    plt.ylabel('true positive rate')
    plt.xlim(0,)
    plt.ylim(0,)
    plt.savefig(str(Path(f'./xgboost-models/{feature_str}-{label_str}-xgboost-model-roc-curve.png')))